In [2]:
import pandas as pd
import os

## Cilj:
Analizirati naslednje:
    -pri kateri starosti šahisti dosežejo svoj največji ELO
    -vpliv nacionalnosti na uspeh
    -ali lahko glede na performance nižjih 100 določimo kdo bo prišel v višjih 100

Zanima nas pri kateri starosti šahisti dosežejo svoj vrhunec. Postopek, da pridobimo to informacijo bo naslednji:
    1)Ustvarili bomo kopijo seznama sahisti ter v to kopijo shranili podatke o tem pri kateri starosti šahist doseže svoj največji ELO rating; podobno bomo naredili tudi z podatkom pri kateri starosti šahist doseže svoj največji indeks na lestvici najboljših 100 šahistov. Ta podatka nista nujno enakovredna, zaradi rating inflationa (https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details)
    2)Statistično analizirali to - verjetno z preprosto aritmetično sredino.
    3) Kreacija grafikonov

In [4]:
pot = r'C:\Users\hugot\Documents\FMF\1_letnik\UVP-projektna-naloga\podatkovna_baza'

sahisti_elo = []

for filename in os.listdir(pot):
    ime_sahista = filename.replace('_', ' ')
    trenutni_maksimum = [(0,0)]
    datotecna_pot = os.path.join(pot, filename)
    print(datotecna_pot, ime_sahista)
    podatki = pd.read_csv(datotecna_pot, index_col = 'datum')
    for datum, vrstica in podatki.iterrows():
        rating = vrstica['rating']
        if rating > trenutni_maksimum[0][1]:
            trenutni_maksimum = [(datum, rating)]
        #naslednja vrstica poskrbi, da ne zgrešimo, če šahist doseže svoj najvišji 
        #rating več kot enkrat.
        elif rating == trenutni_maksimum[0][1]: 
            trenutni_maksimum.append((datum, rating))
    sahisti_elo.append({ime_sahista: trenutni_maksimum})
#sedaj naj bi dobili seznam slovarjev, kjer so kljuci imena sahisov, 
#vrednosti pa seznami naborov (datum, rating) kjer je rating sahistov najvisji rating, 
#datum pa eden izmed datumov, na katerega je slednjega dosegel

print(sahisti_elo)

C:\Users\hugot\Documents\FMF\1_letnik\UVP-projektna-naloga\podatkovna_baza\Abasov,_Nijat


KeyError: 'rating'